<a href="https://colab.research.google.com/github/JanXanevo/CartAnalysis/blob/main/01_snowflakeImport_03_11_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snowflake-sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.24
    Uninstalling SQLAlchemy-2.0.24:
      Successfully uninstalled SQLAlchemy-2.0.24
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.51 which is incompatible.


In [ ]:
import pandas as pd

In [ ]:
from sqlalchemy import create_engine

# snowflake login connection
SNOWFLAKE_USER = 'GUGELM'
SNOWFLAKE_PASSWORD = 'pGtYSXfdP9hJTbP'
SNOWFLAKE_ACC_ID = 'xt02912.europe-west4.gcp'
SNOWFLAKE_DB = 'DC_BI_DB'
SNOWFLAKE_SCHEMA = 'RPT_PRODUCTCREATION'
SNOWFLAKE_ROLE = 'ROLE_PT_PRODUCTCREATION'
SNOWFLAKE_WH = 'COMPUTE_WH'

 #Create the Snowflake connection string
connection_string = f'snowflake://{SNOWFLAKE_USER}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACC_ID}/{SNOWFLAKE_DB}/{SNOWFLAKE_SCHEMA}?role={SNOWFLAKE_ROLE}&warehouse={SNOWFLAKE_WH}'

 # Create the SQLAlchemy engine
engine = create_engine(connection_string)

 # Test the connection
connection = engine.connect()

sql_query = """
SELECT HYBRIS_BESTELLNUMMER, PRODUKTNR, BUSERPK, VERKAUFSBELEG_DATUM, MENGE
FROM "DC_BI_DB"."RPT_PRODUCTCREATION"."BASKETS_RECEIPTS_ONLINE_V"
WHERE BUSERPK IS NOT NULL
AND MENGE > 0
QUALIFY HYBRIS_BESTELLNUMMER = LAG(HYBRIS_BESTELLNUMMER) OVER (ORDER BY HYBRIS_BESTELLNUMMER)
OR HYBRIS_BESTELLNUMMER = LEAD(HYBRIS_BESTELLNUMMER) OVER (ORDER BY HYBRIS_BESTELLNUMMER)
ORDER BY HYBRIS_BESTELLNUMMER
"""

 # Perform SQL operations
onlineBaskets = connection.execute(sql_query)


# Close the connection when done
connection.close()

<ipython-input-3-c0397d876b95>:32: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  onlineBaskets = connection.execute(sql_query)


In [ ]:
dfOnlineBaskets = pd.DataFrame(onlineBaskets)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

df_path ='/content/drive/My Drive/BAHAG/Data/onlineProducts.csv'
dfOnlineBaskets.to_csv(df_path, index=False)

Mounted at /content/drive
